In [ ]:
from casadi import *
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation, rc
from IPython.display import HTML
from scipy.optimize import root

In [ ]:
def energy(q, v):
    u = -np.cos(q)
    t = 0.5 * v**2
    return u+t

In [ ]:
def timecrossings(arr, times, pos):
    crossings = []
    sign_arr = sign(arr)
    for ii in range(1,len(arr)):
        if sign_arr[ii-1] != sign_arr[ii]:
            if cos(pos[ii]) > -0.99 :
                crossings.append((times[ii-1] + times[ii])/2)
    return crossings

In [ ]:
def plot_results(arr, U, T, max_par, N, figsize=[10,7]):
    margin_ang = np.arcsin(max_par)
    timescale_x = np.linspace(0, T, N+1)
    timescale_u = np.linspace(0, T, N)
    #arr_u = sol.value(U)
    arr_u = np.array(U)
    energ = energy(arr[:,0],arr[:,1])
    plt.figure(figsize=figsize)
    plt.plot(timescale_x,arr[:,0], label = '$x$')
    plt.plot(timescale_x,arr[:,1], label = "$x'$")
    plt.plot(timescale_x,energ, label = "$energy$")
    plt.plot(timescale_u,arr_u[:], label = 'u')
    plt.plot(timescale_x,2+cos(arr[:,0]),':', label = '$2+cos(x)$')
    cross_points = timecrossings(arr[:,1], timescale_x, arr[:,0])
    plt.hlines([0,pi, -pi], 0, timescale_u[-1], 'k', 'dotted')
    plt.hlines([-max_par,max_par], 0, timescale_u[-1], 'r', 'dotted')
    plt.hlines([np.pi-margin_ang,-np.pi+margin_ang], 0, timescale_u[-1], 'g', 'dotted', label = "g_par = max par")
    plt.vlines(cross_points, -pi, pi, 'k', 'dotted')
    print(f'Max par: {max_par}, number of crossing points: {len(cross_points)}')
    plt.legend()

In [ ]:
def print_phase_space(X = np.array([[0,0]])):
    X = np.array(X)
    margin = 0.2
    max_x = max(np.max(X[:,0]), np.pi) + margin
    min_x = min(np.min(X[:,0]), -np.pi) - margin
    max_v = max(np.max(X[:,1]), 2) + margin
    min_v = min(np.min(X[:,1]), -2) - margin
    
    x_arr = np.linspace(min_x, max_x, 400)
    v_arr = np.linspace(min_v, max_v, 400)
    
    xx, vv = np.meshgrid(x_arr, v_arr)
    ee = energy(xx,vv)
    
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.contour(xx,vv,ee, 10)
    ax.plot(X[:,0], X[:,1], 'r', marker = 'o', markersize = 5, linewidth = 0.7)
    ax.set_aspect('equal')
    plt.xlabel('Angle in rad')
    plt.ylabel('Angular speed in rad/s')
    plt.title('Phase space, lines of constant energy')

In [ ]:
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 200

In [ ]:
def create_anim(arr, U):
    arr = np.array(arr)
    U = np.array(U)
    fig, (ax, ax2) = plt.subplots(1,2, figsize = [15,8])

    ax.set_aspect('equal')
    ax.set_xlim(( -1.5, 1.5))
    ax.set_ylim(( -1.5, 1.5))

    circle2 = plt.Circle((0, 0), 1, color='b', ls = ":", fill=False)
    ax.add_artist(circle2)

    line, = ax.plot([], [], lw=2)
    point, = ax.plot([], [], marker='o', markersize=15, color="red")
    text = ax.text(0.2, 0, "", fontsize = 12)
    text_2 = ax.text(0.2, -0.15, "", fontsize = 12)
    text_3 = ax.text(0.2, -0.3, "", fontsize = 12)
    # Phase space
    
    margin = 0.2
    max_x = max(np.max(arr[:,0]), np.pi) + margin
    min_x = min(np.min(arr[:,0]), -np.pi) - margin
    max_v = max(np.max(arr[:,1]), 2) + margin
    min_v = min(np.min(arr[:,1]), -2) - margin
    
    x_arr = np.linspace(min_x, max_x, 400)
    v_arr = np.linspace(min_v, max_v, 400)
    
    xx, vv = np.meshgrid(x_arr, v_arr)
    ee = energy(xx,vv)
    
    ax2.contour(xx,vv,ee, 10)
    line2, = ax2.plot([], [], 'r', marker = 'o', markersize = 5, linewidth = 0.7)
    ax2.set_aspect('equal')
    ax2.set_xlabel('Angle in rad')
    ax2.set_ylabel('Angular speed in rad/s')
    ax2.set_title('Phase space, lines of constant energy')
    
    trayectory = np.zeros_like(arr)
    trayectory[:,0] = arr[0,0]
    trayectory[:,1] = arr[0,1]
    
    def init():
        line.set_data([], [])
        point.set_data([], [])
        text.set_text('')
        return (line,)
    def animate(i):
        x = [0, np.sin(arr[i,0])]
        y = [0, -np.cos(arr[i,0])]
        line.set_data(x, y)    
        point.set_data(x[1], y[1])
        text.set_text("U = %.6f" % U[i])
        text_2.set_text(r"$\dot{\theta}$" + " = %.6f" % arr[i,1])
        text_3.set_text("par g = %.6f" % x[1])
        
        trayectory[i:,0] = arr[i,0]
        trayectory[i:,1] = arr[i,1]
        line2.set_data(trayectory[:,0], trayectory[:,1])
        return (line, line2,)
    
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=arr.shape[0]-1, interval=20, 
                               blit=True)
    return anim

## Pendulum exercise
$$
\begin{split}\begin{array}{lc}
\begin{array}{l}
\text{minimize:} \\
x(\cdot) \in \mathbb{R}^2, \, u(\cdot) \in \mathbb{R}
\end{array}
\quad \displaystyle \int_{t=0}^{T}{cos(x_0) \, dt}
\\
\\
\text{subject to:} \\
\\
\begin{array}{ll}
\left\{
\begin{array}{l}
\dot{x}_0 = x_1 \\
\dot{x}_1 = u - sin(x_0) \\
-u_{max} \le u \le u_{max} , \quad
\end{array} \right. & \text{for} \, 0 \le t \le T \\
x_0(0)=0, \quad x_1(0)=0, x_0(T) = pi/2 , x_1(T) = 0
\end{array}
\end{array}\end{split}
$$
with $T=10$.

siendo $$x_0 = \theta$$ $$x_1 = \theta'$$

In [ ]:
x = MX.sym('x', 2)
t = MX.sym('t')
dt = MX.sym('dt')
u = MX.sym('u')

rhs = vertcat(x[1], u-sin(x[0]))
#rhs = vertcat(x[1], u)
F = Function('F', [x, u], [rhs])

k1 = F(x, u);
k2 = F(x + dt/2 * k1, u)
k3 = F(x + dt/2 * k2, u)
k4 = F(x + dt * k3, u);
new_x_expr = x+dt/6*(k1 +2*k2 +2*k3 +k4)

new_x_expr = x + dt * F(x, u)
new_x = Function('New_x', [x, u, dt], [new_x_expr])

## Opti problem

In [ ]:
N = 200

In [ ]:
opti = Opti()
opti.solver('ipopt')

In [ ]:
X = opti.variable(N+1,2)
U = opti.variable(N)
#T = opti.variable()
T = opti.parameter()
u_m = opti.parameter()
#t_m = opti.parameter()

In [ ]:
cost = sum1(2+cos(X[:,0]))*T #**2
#cost = -sum1(X[:,0])
opti.minimize(cost)

In [ ]:
opti.subject_to(X[0,:].T == [0, 0])
opti.subject_to(cos(X[-1,0]) < -0.9999)
#opti.subject_to(T < t_m)
opti.subject_to(opti.bounded(-0.001,X[-1,1],0.001))

In [ ]:
for ii in range(N):
    opti.subject_to(X[ii+1,:].T == new_x(X[ii,:], U[ii], T/N))
    opti.subject_to(opti.bounded(-u_m,U[ii],u_m))

In [ ]:
opti.set_initial(X[:,0], np.linspace(0, pi, N+1))
opti.set_initial(X[:,1], pi/N)
#opti.set_initial(T, 50)
opti.set_value(T, 25)
max_par = 0.2
opti.set_value(u_m, max_par)

In [ ]:
sol = opti.solve()

In [ ]:
plot_results(sol.value(X), sol.value(U), sol.value(T), max_par, N)

In [ ]:
#anim = create_anim(sol)

In [ ]:
#HTML(anim.to_jshtml())

In [ ]:
results = []
for ii in [0.11, 0.105, 0.1, 0.095, 0.09, 0.085]:
    opti.set_value(u_m, ii)
    est_t = (2 + sqrt(1/ii))*8
    #opti.set_initial(T, est_t)
    opti.set_value(T, 40)
    try:
        sol = opti.solve()
    except:
        pass
    else:
        results.append([sol.value(X), sol.value(U), sol.value(T), ii, N])

In [ ]:
for res in results:
    plot_results(*res)

## Integración de la acción para comprobación

In [ ]:
def euler_step(x, u, dt):
    return x + dt * F(x, u)

In [ ]:
def rk4_step(x, u, dt):
    k1 = F(x, u);
    k2 = F(x + dt/2 * k1, u)
    k3 = F(x + dt/2 * k2, u)
    k4 = F(x + dt * k3, u);
    return x+dt/6*(k1 +2*k2 +2*k3 +k4)

In [ ]:
def integrate_euler(x_0, u, dt):
    x = [x_0,]
    for ii in range(len(u)):
        x_i = euler_step(x[-1], u[ii], dt)
        x.append(x_i)
    return horzcat(*x).T

In [ ]:
def integrate_rk4(x_0, u, dt):
    x = [x_0,]
    for ii in range(len(u)):
        x_i = rk4_step(x[-1], u[ii], dt)
        x.append(x_i)
    return horzcat(*x).T

In [ ]:
def plot_results_sim(U, arr, T, max_par, N):
    arr = np.array(arr)
    timescale_x = np.linspace(0, T, N+1)
    timescale_u = np.linspace(0, T, N)
    arr_u = np.array(U)
    energ = energy(arr[:,0],arr[:,1])
    plt.figure(figsize=[10,7])
    plt.plot(timescale_x,arr[:,0], label = '$x$')
    plt.plot(timescale_x,arr[:,1], label = "$x'$")
    plt.plot(timescale_x,energ, label = "$energy$")
    plt.plot(timescale_u,arr_u[:], label = 'u')
    plt.plot(timescale_x,2+cos(arr[:,0]),':', label = '$2+cos(x)$')
    cross_points = timecrossings(arr[:,1], timescale_x, arr[:,0])
    plt.hlines([0,pi, -pi], 0, timescale_u[-1], 'k', 'dotted')
    plt.hlines([-max_par,max_par], 0, timescale_u[-1], 'r', 'dotted')
    plt.vlines(cross_points, -pi, pi, 'k', 'dotted')
    print(f'Max par: {max_par}, number of crossing points: {len(cross_points)}')
    plt.legend()

## Comparación entre integrar con Euler y Runge Kutta

In [ ]:
x_0 = DM([1,0])
u_0 = np.zeros(200)
xx = integrate_euler(x_0, u_0, 0.1)
print_phase_space(xx)

In [ ]:
xx = integrate_rk4(x_0, u_0, 0.1)
print_phase_space(xx)

### Usando Euler:

In [ ]:
xx = integrate_euler(DM([0,0]), sol.value(U), sol.value(T)/N)
plot_results_sim(sol.value(U), xx, sol.value(T), sol.value(max_par), N)

### Usando Runge Kutta

In [ ]:
xx = integrate_rk4(DM([0,0]), sol.value(U), sol.value(T)/N)
plot_results_sim(sol.value(U), xx, sol.value(T), sol.value(max_par), N)

## Otras comprobaciones: Coste = T

In [ ]:
N = 300

k1 = F(x, u);
k2 = F(x + dt/2 * k1, u)
k3 = F(x + dt/2 * k2, u)
k4 = F(x + dt * k3, u);
new_x_expr = x+dt/6*(k1 +2*k2 +2*k3 +k4)

new_x_expr = x + dt * F(x, u)
new_x = Function('New_x', [x, u, dt], [new_x_expr])

opti = Opti()
opti.solver('ipopt')

X = opti.variable(N+1,2)
U = opti.variable(N)
T = opti.variable()
u_m = opti.parameter()
#t_m = opti.parameter()

#cost = sum1(2+cos(X[:,0]))*T #**2
cost = T
#cost = -sum1(X[:,0])
opti.minimize(cost)

opti.subject_to(X[0,:].T == [0, 0])
opti.subject_to(cos(X[-1,0]) < -0.9999)
#opti.subject_to(T < t_m)
opti.subject_to(opti.bounded(-0.001,X[-1,1],0.001))

for ii in range(N):
    opti.subject_to(X[ii+1,:].T == new_x(X[ii,:], U[ii], T/N))
    opti.subject_to(opti.bounded(-u_m,U[ii],u_m))

In [ ]:
opti.set_initial(X[:,0], np.linspace(0, pi, N+1))
opti.set_initial(X[:,1], pi/N)
opti.set_initial(T, 50)
max_par = 0.1
opti.set_value(u_m, max_par)

sol = opti.solve()

In [ ]:
plot_results(sol.value(X), sol.value(U),sol.value(T), max_par, N)

In [ ]:
xx = integrate_euler(DM([0,0]), sol.value(U), sol.value(T)/N)
plot_results_sim(sol.value(U), xx, sol.value(T), sol.value(max_par), N)

In [ ]:
#anim = create_anim(sol)

In [ ]:
#HTML(anim.to_jshtml())

## Otras comprobaciones: Alimentar integradores avanzados con solución de Euler

In [ ]:
N = 500

k1 = F(x, u);
k2 = F(x + dt/2 * k1, u)
k3 = F(x + dt/2 * k2, u)
k4 = F(x + dt * k3, u);
new_x_expr_rk = x+dt/6*(k1 +2*k2 +2*k3 +k4)

new_x_expr_eu = x + dt * F(x, u)

new_x_expr_mid = x + dt * F(x + 0.5*dt*F(x, u), u)

new_x_eu = Function('New_x_eu', [x, u, dt], [new_x_expr_eu])
new_x_mid = Function('New_x_mid', [x, u, dt], [new_x_expr_mid])
new_x_rk = Function('New_x_rk', [x, u, dt], [new_x_expr_rk])

In [ ]:
def create_pend(new_x):
    opti = Opti()
    opti.solver('ipopt')

    X = opti.variable(N+1,2)
    U = opti.variable(N)
    T = opti.parameter()
    u_m = opti.parameter()
    #t_m = opti.parameter()

    cost = sum1(cos(X[:,0])) #**2
    #cost = T
    #cost = -sum1(X[:,0])
    opti.minimize(cost)

    opti.subject_to(X[0,:].T == [0, 0])
    

    for ii in range(N):
        opti.subject_to(X[ii+1,:].T == new_x(X[ii,:], U[ii], T/N))
        opti.subject_to(opti.bounded(-u_m,U[ii],u_m))
    return opti, X, U, T, u_m

In [ ]:
opti_eu, X, U, T, u_m = create_pend(new_x_eu)

opti_eu.subject_to(cos(X[-1,0]) < -0.9999)
#opti.subject_to(T < t_m)
opti_eu.subject_to(opti.bounded(-0.001,X[-1,1],0.001))
    
opti_eu.set_initial(X[:,0], np.linspace(0, pi, N+1))
opti_eu.set_initial(X[:,1], pi/N)
opti_eu.set_value(T, 60)
max_par = 0.1
opti_eu.set_value(u_m, max_par)

sol_eu = opti_eu.solve()

In [ ]:
X_eu = sol_eu.value(X)
U_eu = sol_eu.value(U)
T_eu = sol_eu.value(T)

In [ ]:
plot_results(X_eu, U_eu, T_eu, max_par, N)

In [ ]:
xx = integrate_rk4(DM([0,0]), U_eu, T_eu/N)
plot_results_sim(U_eu, xx, T_eu, 0.1, N)

In [ ]:
print_phase_space(X_eu)

In [ ]:
opti_mid, X, U, T, u_m = create_pend(new_x_mid)
opti_mid.set_initial(X, X_eu)
#opti_mid.set_value(T, T_eu)
opti_mid.set_value(T, 120)
opti_mid.set_initial(U, U_eu)
max_par = 0.1
opti_mid.set_value(u_m, max_par)

sol_mid = opti_mid.solve()

In [ ]:
X_mid = sol_mid.value(X)
U_mid = sol_mid.value(U)
T_mid = sol_mid.value(T)

In [ ]:
plot_results(X_mid, U_mid, T_mid, max_par, N)

In [ ]:
xx = integrate_rk4(DM([0,0]), U_mid, T_mid/N)
plot_results_sim(U_mid, xx, T_mid, 0.1, N)

In [ ]:
opti_rk, X, U, T, u_m = create_pend(new_x_rk)
opti_rk.set_initial(X, X_mid)
opti_rk.set_value(T, T_mid)
opti_rk.set_initial(U, U_mid)
max_par = 0.1
opti_rk.set_value(u_m, max_par)

sol_rk = opti_rk.solve()

In [ ]:
X_rk = sol_rk.value(X)
U_rk = sol_rk.value(U)
T_rk = sol_rk.value(T)

In [ ]:
plot_results(X_rk, U_rk, T_rk, max_par, N)

In [ ]:
xx = integrate_rk4(DM([0,0]), U_rk, T_rk/N)
plot_results_sim(U_rk, xx, T_rk, 0.1, N)

In [ ]:
print_phase_space(xx)

anim = create_anim(X_rk, U_rk)

HTML(anim.to_jshtml())

## New Trapezoidal scheme

In [ ]:
def trapz_opti_step(x_n, x, u, u_n, dt):
    f = np.zeros_like(x)
    f[0] = x[1]
    f[1] = u-np.sin(x[0])
    f_n = np.zeros_like(x)
    f_n[0] = x_n[1]
    f_n[1] = u_n-np.sin(x_n[0])
    res = x + dt/2*(f+f_n) - x_n
    return res

def trapz_step(x, u, u_n, dt):
    x_n = root(trapz_opti_step, x, (x, u, u_n, dt))
    return x_n.x

def integrate_trapz(x_0, u, dt):
    x = [x_0,]
    u = np.append(u, u[-1])
    for ii in range(0,len(u)-1):
        x_i = trapz_step(x[-1], u[ii], u[ii+1], dt)
        x.append(x_i)
    return horzcat(*x).T

In [ ]:
q = MX.sym('q')
v = MX.sym('v')
t = MX.sym('t')
dt = MX.sym('dt')
u = MX.sym('u')

rhs = u-sin(q)
#rhs = vertcat(x[1], u)
F1d = Function('F', [q, u], [rhs])

#k1 = F(x, u);
#k2 = F(x + dt/2 * k1, u)
#k3 = F(x + dt/2 * k2, u)
#k4 = F(x + dt * k3, u);
#new_x_expr = x+dt/6*(k1 +2*k2 +2*k3 +k4)
q_n = MX.sym('q_n')
u_n = MX.sym('u_n')
v_n_expr = v + (1/2) * dt * (F1d(q_n,u_n) + F1d(q,u))
q_n_expr = q + v * dt + (1/6) * dt**2 * (F1d(q_n,u_n) + 2*F1d(q,u))

new_q = Function('New_q', [q, q_n, v, u, u_n, dt], [q_n_expr])
new_v = Function('New_v', [q, q_n, v, u, u_n, dt], [v_n_expr])

In [ ]:
opti = Opti()
opti.solver('ipopt')

Q = opti.variable(N+1)
V = opti.variable(N+1)
U = opti.variable(N+1)
T = opti.parameter()
u_m = opti.parameter()
#t_m = opti.parameter()

cost = sum1(cos(Q)) #**2
#cost = T
#cost = -sum1(X[:,0])
opti.minimize(cost)

opti.subject_to(Q[0] == 0)
opti.subject_to(V[0] == 0)
opti.subject_to(V[-1] == 0)


for ii in range(N):
    opti.subject_to(Q[ii+1] == new_q(Q[ii], Q[ii+1], V[ii], U[ii], U[ii+1], T/N))
    opti.subject_to(V[ii+1] == new_v(Q[ii], Q[ii+1], V[ii], U[ii], U[ii+1], T/N))
    opti.subject_to(opti.bounded(-u_m,U[ii],u_m))


In [ ]:
opti.set_initial(Q, np.linspace(0, pi, N+1))
opti.set_initial(V, pi/N)
#opti.set_initial(T, 50)
opti.set_value(T, 25)
max_par = 0.2
opti.set_value(u_m, max_par)

In [ ]:
sol = opti.solve()

In [ ]:
X_sol = np.zeros([N+1,2])
X_sol[:,0] = sol.value(Q)
X_sol[:,1] = sol.value(V)
plot_results(X_sol, sol.value(U)[:-1], sol.value(T), max_par, N)

In [ ]:
print_phase_space(X_sol)

anim = create_anim(X_sol, sol.value(U)[:-1])

HTML(anim.to_jshtml())

In [ ]:
xx = integrate_rk4(DM([0,0]), sol.value(U)[:-1], sol.value(T)/N)
plot_results_sim(sol.value(U)[:-1], xx, sol.value(T), 0.1, N)

In [ ]:
print_phase_space(xx)

## Energy test

In [ ]:
import time

In [ ]:
def pend_RK4(q0 = 1, T = 20, N = 50):
    t0 = time.time()
    xx = integrate_rk4(np.array([q0,0.]), np.zeros(N), T/N)
    t1 = time.time()
    return np.array(xx), np.zeros(N), t1-t0

In [ ]:
def pend_trapz(q0 = 1, T = 20, N = 50):
    t0 = time.time()
    xx = integrate_trapz(np.array([q0,0.]), np.zeros(N), T/N)
    t1 = time.time()
    return np.array(xx), np.zeros(N), t1-t0

### Trapezoidal implícita normal para comparar

In [ ]:
q = MX.sym('q')
v = MX.sym('v')
t = MX.sym('t')
dt = MX.sym('dt')
u = MX.sym('u')

rhs = u-sin(q)
F1d = Function('F', [q, u], [rhs])


q_n = MX.sym('q_n')
u_n = MX.sym('u_n')
v_n = MX.sym('v_n')
v_n_expr = v + (1/2) * dt * (F1d(q_n,u_n) + F1d(q,u))
q_n_expr = q + (1/2) * dt * (v + v_n)

new_q_tr = Function('New_q', [q, v, v_n, u, u_n, dt], [q_n_expr])
new_v_tr = Function('New_v', [q, q_n, v, u, u_n, dt], [v_n_expr])

In [ ]:
q = MX.sym('q')
v = MX.sym('v')
t = MX.sym('t')
dt = MX.sym('dt')
u = MX.sym('u')

rhs = u-sin(q)
F1d = Function('F', [q, u], [rhs])


q_n = MX.sym('q_n')
u_n = MX.sym('u_n')
v_n = MX.sym('v_n')
v_n_expr = v + (1/2) * dt * (F1d(q_n,u_n) + F1d(q,u))
q_n_expr = q + v * dt + (1/4) * dt**2 * (F1d(q_n,u_n) + F1d(q,u))

new_q_tr_2 = Function('New_q', [q, q_n, v, u, u_n, dt], [q_n_expr])
new_v_tr_2 = Function('New_v', [q, q_n, v, u, u_n, dt], [v_n_expr])

In [ ]:
def pend_trapz_casadi(q0 = 1, T = 20, N = 50):
    opti = Opti()
    opti.solver('ipopt')

    Q = opti.variable(N+1)
    V = opti.variable(N+1)
    U = opti.variable(N+1)
    u_m = opti.parameter()

    cost = sum1(U**2) 
    opti.minimize(cost)

    opti.subject_to(Q[0] == q0)
    opti.subject_to(V[0] == 0)


    for ii in range(N):
        opti.subject_to(Q[ii+1] == new_q_tr(Q[ii], V[ii], V[ii+1], U[ii], U[ii+1], T/N))
        opti.subject_to(V[ii+1] == new_v_tr(Q[ii], Q[ii+1], V[ii], U[ii], U[ii+1], T/N))
        opti.subject_to(opti.bounded(-u_m,U[ii],u_m))
        
    xx = integrate_rk4(DM([q0,0]), np.zeros(N), T/N)
    opti.set_initial(Q, xx[:,0])
    opti.set_initial(V, xx[:,1])
    opti.set_initial(U, 0)
    max_par = 0.0
    opti.set_value(u_m, max_par)
    
    t0 = time.time()
    sol_tr = opti.solve()
    t1 = time.time()
    
    X_sol_tr = np.zeros([N+1,2])
    X_sol_tr[:,0] = sol_tr.value(Q)
    X_sol_tr[:,1] = sol_tr.value(V)
    return X_sol_tr, sol_tr.value(U), t1-t0

In [ ]:
def pend_trapz_casadi_2(q0 = 1, T = 20, N = 50):
    opti = Opti()
    opti.solver('ipopt')

    Q = opti.variable(N+1)
    V = opti.variable(N+1)
    U = opti.variable(N+1)
    u_m = opti.parameter()

    cost = sum1(U**2) 
    opti.minimize(cost)

    opti.subject_to(Q[0] == q0)
    opti.subject_to(V[0] == 0)


    for ii in range(N):
        opti.subject_to(Q[ii+1] == new_q_tr_2(Q[ii], Q[ii+1], V[ii], U[ii], U[ii+1], T/N))
        opti.subject_to(V[ii+1] == new_v_tr_2(Q[ii], Q[ii+1], V[ii], U[ii], U[ii+1], T/N))
        opti.subject_to(opti.bounded(-u_m,U[ii],u_m))
        
    xx = integrate_rk4(DM([q0,0]), np.zeros(N), T/N)
    opti.set_initial(Q, xx[:,0])
    opti.set_initial(V, xx[:,1])
    opti.set_initial(U, 0)
    max_par = 0.0
    opti.set_value(u_m, max_par)
    
    t0 = time.time()
    sol_tr = opti.solve()
    t1 = time.time()
    
    X_sol_tr = np.zeros([N+1,2])
    X_sol_tr[:,0] = sol_tr.value(Q)
    X_sol_tr[:,1] = sol_tr.value(V)
    return X_sol_tr, sol_tr.value(U), t1-t0

In [ ]:
T_test = 120
N_test = 100

In [ ]:
X_sol_tr, U_sol_tr, _ = pend_trapz_casadi(q0 = 2, T = T_test, N = N_test)

In [ ]:
plot_results(X_sol_tr, U_sol_tr[:-1], T_test, 0, N_test)
plt.title('trapezoidal clásico')

In [ ]:
X_sol_tr, U_sol_tr, _ = pend_trapz(q0 = 2, T = T_test, N = N_test)

In [ ]:
plot_results(X_sol_tr, U_sol_tr, T_test, 0, N_test)
plt.title('trapezoidal clásico')

### Trapezoidal modificada

In [ ]:
def trapz_mod_opti_step(x_n, x, u, u_n, dt):
    f = u-np.sin(x[0])
    f_n = u_n-np.sin(x_n[0])
    res = np.zeros_like(x_n)
    res[1] = x[1] + dt/2*(f+f_n) - x_n[1]
    res[0] = x[0] + dt * x[1] + dt**2/6*(f_n + 2*f) - x_n[0]
    return res

def trapz_mod_step(x, u, u_n, dt):
    x_n = root(trapz_mod_opti_step, x, (x, u, u_n, dt))
    return x_n.x

def integrate_trapz_mod(x_0, u, dt):
    x = [x_0,]
    u = np.append(u, u[-1])
    for ii in range(len(u)-1):
        x_i = trapz_mod_step(x[-1], u[ii], u[ii+1], dt)
        x.append(x_i)
    return horzcat(*x).T

def pend_trapz_mod(q0 = 1, T = 20, N = 50):
    t0 = time.time()
    xx = integrate_trapz_mod(np.array([q0,0.]), np.zeros(N), T/N)
    t1 = time.time()
    return np.array(xx), np.zeros(N), t1-t0

In [ ]:
_ = trapz_mod_step(np.array([1.,0.]), 0,0,0.3)
_

In [ ]:
trapz_mod_opti_step(_, np.array([1.,0.]), 0,0,0.3)

In [ ]:
def pend_trapz_mod_casadi(q0 = 1, T = 20, N = 50):
    opti = Opti()
    opti.solver('ipopt')

    Q = opti.variable(N+1)
    V = opti.variable(N+1)
    U = opti.variable(N+1)
    u_m = opti.parameter()

    cost = sum1(U**2) 
    opti.minimize(cost)

    opti.subject_to(Q[0] == q0)
    opti.subject_to(V[0] == 0)


    for ii in range(N):
        opti.subject_to(Q[ii+1] == new_q(Q[ii], Q[ii+1], V[ii], U[ii], U[ii+1], T/N))
        opti.subject_to(V[ii+1] == new_v(Q[ii], Q[ii+1], V[ii], U[ii], U[ii+1], T/N))
        opti.subject_to(opti.bounded(-u_m,U[ii],u_m))
        
    xx = integrate_rk4(DM([q0,0]), np.zeros(N), T/N)
    opti.set_initial(Q, xx[:,0])
    opti.set_initial(V, xx[:,1])
    opti.set_initial(U, 0)
    max_par = 0.0
    opti.set_value(u_m, max_par)
    
    t0 = time.time()
    sol_tr = opti.solve()
    t1 = time.time()
    
    X_sol_tr = np.zeros([N+1,2])
    X_sol_tr[:,0] = sol_tr.value(Q)
    X_sol_tr[:,1] = sol_tr.value(V)
    return X_sol_tr, sol_tr.value(U), t1-t0

In [ ]:
X_sol, U_sol, _ = pend_trapz_mod_casadi(q0 = 2, T = T_test, N = N_test)

In [ ]:
plot_results(X_sol, U_sol[:-1], T_test, 0, N_test)
plt.title('trapezoidal modificada')

In [ ]:
X_sol, U_sol, _ = pend_trapz_mod(q0 = 2, T = T_test, N = N_test)

In [ ]:
plot_results(X_sol, U_sol, T_test, 0, N_test)
plt.title('trapezoidal modificada')

In [ ]:
print_phase_space(X_sol)

In [ ]:
xx = integrate_rk4(DM([2,0]), U_sol, T_test/N_test)
plot_results_sim(U_sol, xx, T_test, 0.0, N_test)
plt.title('Propagando RK4')

## Comparing Errors

In [ ]:
from scipy.interpolate import CubicHermiteSpline as hermite

In [ ]:
def new_trap_interp_q(q, q_n, v, u, u_n, tau, h):
    res = q + v * tau + 1/2 * F1d(q,u) * tau**2 + 1/(6*h) * tau**3 * (F1d(q_n,u_n) - F1d(q,u))
    return res
def trap_interp_q(q, v, v_n, tau, h):
    res = q + v * tau + 1/(2*h) * tau**2 * (v_n - v)
    return res
def new_trap_interp_v(q, q_n, v, u, u_n, tau, h):
    res = v + tau * F1d(q,u) + tau**2 / (2*h) * (F1d(q_n,u_n) - F1d(q,u))
    return res
def newpoint(Q, V, U, h, t, scheme):
    n = int(t//h)
    tau = t%h
    #print(f't = {t} , tau = {tau} , n = {n} , h = {h}')
    if abs(tau) < h*Q.size*1e-10:
        q = Q[n]
        v = V[n]
    else:
        if scheme == 'mod':
            q = new_trap_interp_q(Q[n], Q[n+1], V[n], U[n], U[n+1], tau, h)
        elif scheme == 'trapz':
            q = trap_interp_q(Q[n], V[n], V[n+1], tau, h)
        else:
            raise NameError(f'scheme {scheme} not recognized')
        v = new_trap_interp_v(Q[n], Q[n+1], V[n], U[n], U[n+1], tau, h)
    return q,v
def newtrap_interpolated_array(Q, V, U, h, t_array, scheme = 'mod'):
    N = t_array.size
    new_Q = np.zeros(N)
    new_V = np.zeros(N)
    if Q.size != U.size:
        U = np.append(U, U[-1])
    old_t_array = np.linspace(0, (Q.size-1)*h, Q.size)
    #print(Q.size, U.size, old_t_array.size, Q, U, old_t_array)
    new_U = np.interp(t_array, old_t_array, U)
    if scheme == 'hermite':
        q_interp = hermite(old_t_array, Q, V)
        new_Q = q_interp(t_array)
        A = U-np.sin(Q)
        v_interp = hermite(old_t_array, V, A)
        new_V = v_interp(t_array)
    else:
        for ii in range(N):
            new_Q[ii], new_V[ii] = newpoint(Q, V, U, h, t_array[ii], scheme)
    return new_Q, new_V, new_U

In [ ]:
plot_results(X_sol[:22,:], U_sol[:21], T_test*21/N_test, 0, 21)

In [ ]:
N_interp = 400
t_array_interp = np.linspace(0, 20, N_interp)
new_Q, new_V, new_U = newtrap_interpolated_array(X_sol[:,0], X_sol[:,1], U_sol,
                                                 T_test/N_test, t_array_interp, 'mod')
X_interp = np.zeros([N_interp,2])
X_interp[:,0] = new_Q
X_interp[:,1] = new_V
plot_results(X_interp, new_U[:-1], t_array_interp[-1], max_par, N_interp-1, figsize = [20,15])
plt.plot(np.linspace(0, T_test, N_test+1)[:17], X_sol[:,0][:17], 'k.')
plt.plot(np.linspace(0, T_test, N_test+1)[:17], X_sol[:,1][:17], 'k.')

## Comparación con trapezoidal normal

In [ ]:
N_interp = 400
t_array_interp = np.linspace(0, 20, N_interp)
new_Q, new_V, new_U = newtrap_interpolated_array(X_sol_tr[:,0], X_sol_tr[:,1], U_sol_tr,
                                                 T_test/N_test, t_array_interp, 'trapz')
X_interp_tr = np.zeros([N_interp,2])
X_interp_tr[:,0] = new_Q
X_interp_tr[:,1] = new_V
plot_results(X_interp_tr, new_U[:-1], t_array_interp[-1], max_par, N_interp-1, figsize = [20,15])
plt.plot(np.linspace(0, T_test, N_test+1)[:17], X_sol_tr[:,0][:17], 'k.')
plt.plot(np.linspace(0, T_test, N_test+1)[:17], X_sol_tr[:,1][:17], 'k.')

In [ ]:
X_sol_rk4, U_sol_rk4, _ = pend_RK4(q0 = 2, T = T_test, N = N_test)
new_Q, new_V, new_U = newtrap_interpolated_array(X_sol_rk4[:,0], X_sol_rk4[:,1], U_sol_rk4,
                                                 T_test/N_test, t_array_interp, 'hermite')
X_interp_rk4 = np.zeros([N_interp,2])
X_interp_rk4[:,0] = new_Q
X_interp_rk4[:,1] = new_V
plot_results(X_interp_rk4, new_U[:-1], t_array_interp[-1], max_par, N_interp-1, figsize = [20,15])
plt.plot(np.linspace(0, T_test, N_test+1)[:17], X_sol_rk4[:,0][:17], 'k.')
plt.plot(np.linspace(0, T_test, N_test+1)[:17], X_sol_rk4[:,1][:17], 'k.')

In [ ]:
from scipy.special import ellipj, ellipk

In [ ]:
def exact_pendulum(t, q0 = 1, omega_0 = 1):
    m = np.sin(q0/2)**2
    u = ellipk(m) - omega_0 * t
    return 2 * np.arcsin(np.sin(q0/2) * ellipj(u,m)[0])

In [ ]:
plt.figure(figsize=[15,10])
plt.plot(t_array_interp, exact_pendulum(t_array_interp, 2), 'r', label = 'exact')
plt.plot(t_array_interp, X_interp[:,0], label = 'modified trapezoid')
plt.plot(t_array_interp, X_interp_tr[:,0], label = 'normal trapezoid')
plt.plot(t_array_interp, X_interp_rk4[:,0],  label = 'RK4')
plt.plot(np.linspace(0, T_test, N_test+1)[:17], X_sol_tr[:,0][:17], 'k.')
plt.plot(np.linspace(0, T_test, N_test+1)[:17], X_sol[:,0][:17], 'k.')
plt.plot(np.linspace(0, T_test, N_test+1)[:17], exact_pendulum(np.linspace(0, T_test, N_test+1)[:17], 2), 'k.')
plt.plot(np.linspace(0, T_test, N_test+1)[:17], X_sol_rk4[:,0][:17], 'k.')
plt.legend()
plt.grid()

### Comparando error de los puntos generados
Error cuadrático medio de 100 puntos en función de h

In [ ]:
def plot_comparison(time, Q_exact, Q_trapz, Q_mod, Q_rk4, figsize=[10,7]):
    plt.figure(figsize=figsize)
    plt.plot(time, Q_exact, label = 'exact')
    plt.plot(time, Q_mod, label = 'modified trapezoid')
    plt.plot(time, Q_trapz, label = 'normal trapezoid')
    plt.plot(time, Q_rk4, label = 'RK4')
    plt.legend()
    plt.grid()

In [ ]:
timescales = [0.1, 0.2, 0.5, 0.6, 0.7, 1, 2, 5, 10, 25, 50, 70, 80, 90, 100]

eqm_trapz = []
eqm_mod = []
eqm_rk4 = []

dt_trapz = []
dt_mod = []
dt_rk4 = []

N = 100
q0 = 2
plots = False
for T in timescales:
    X_sol_mod, _ , dt_mod_i = pend_trapz_mod(q0 = q0, T = T, N = N)
    X_sol_tr, _ , dt_trapz_i = pend_trapz(q0 = q0, T = T, N = N)
    X_sol_rk4, _ , dt_rk4_i = pend_RK4(q0 = q0, T = T, N = N)
    Q_exact = exact_pendulum(np.linspace(0, T, N+1), q0)
    
    eqm_trapz.append(np.mean((Q_exact - X_sol_tr[:,0])**2))
    eqm_mod.append(np.mean((Q_exact - X_sol_mod[:,0])**2))
    eqm_rk4.append(np.mean((Q_exact - X_sol_rk4[:,0])**2))
    
    dt_trapz.append(dt_trapz_i)
    dt_mod.append(dt_mod_i)
    dt_rk4.append(dt_rk4_i)
    
    if plots:
        plot_comparison(np.linspace(0,T,N+1),
                   Q_exact,
                   X_sol_tr[:,0],
                   X_sol_mod[:,0],
                   X_sol_rk4[:,0])


In [ ]:
plt.figure(figsize=[15,10])
h_scales = np.array(timescales)/N
plt.plot(h_scales, eqm_trapz, label = 'normal trapezoid')
plt.plot(h_scales, eqm_mod, label = 'modified trapezoid')
plt.plot(h_scales, eqm_rk4, label = 'RK4')
plt.vlines(h_scales, np.min(eqm_rk4)*0.9, np.max(eqm_trapz)*1.1, 'k', 'dotted')
plt.hlines(4, min(h_scales), max(h_scales), 'k', 'dotted')
plt.legend()
plt.grid()
plt.yscale('log')
plt.xscale('log')
plt.xlabel('h')
plt.ylabel('Error cuadrático medio')

In [ ]:
plt.figure(figsize=[15,10])
h_scales = np.array(timescales)/N
plt.plot(h_scales, dt_trapz, label = 'normal trapezoid')
plt.plot(h_scales, dt_mod, label = 'modified trapezoid')
plt.plot(h_scales, dt_rk4, label = 'RK4')
plt.vlines(h_scales, np.min(dt_rk4)*0.9, np.max(dt_trapz)*1.1, 'k', 'dotted')
plt.legend()
plt.grid()
plt.yscale('log')
plt.xscale('log')
plt.xlabel('h')
plt.ylabel('Tiempo de proceso (s)')

## Comparando error de interpolaciones
Interpolando con 200 puntos, generando 15 puntos

In [ ]:
def error_transcr(Q, V, U, h):
    if Q.size != U.size:
        U = np.append(U, U[-1])
    Qdot = (Q[2:]-Q[:-2])/(2*h)
    Vdot = (V[2:]-V[:-2])/(2*h)
    fv_arr = U[1:-1]-np.sin(Q[1:-1])
    eps_Q = np.abs(Qdot - V[1:-1])
    eps_V = np.abs(Vdot- fv_arr)
    return eps_Q, eps_V

def error_transcr_medio(Q, V, U, h):
    eps_Q, eps_V = error_transcr(Q, V, U, h)
    return np.mean(eps_Q + eps_V)

In [ ]:
timescales_int = [0.1, 0.2, 0.5, 1, 2, 5, 7, 8, 9, 10, 15, 17, 20, 22, 25]

eqm_trapz_int = []
eqm_mod_int = []
eqm_rk4_int = []

dt_trapz_int = []
dt_mod_int = []
dt_rk4_int = []

err_trans_trapz = []
err_trans_mod = []
err_trans_rk4 = []

N = 15
q0 = 2
N_interp = 1000
plots = False

for T in timescales_int:
    
    t_array_interp = np.linspace(0, T, N_interp)
    h = t_array_interp[1]-t_array_interp[0]
    
    X_sol_mod, U_sol_mod , dt_trapz_i = pend_trapz_mod(q0 = q0, T = T, N = N)
    Q_mod, V_mod, U_mod = newtrap_interpolated_array(X_sol_mod[:,0], X_sol_mod[:,1], U_sol_mod,
                                                     T/N, t_array_interp, 'mod')
    #print(U_sol_mod)
    X_sol_tr, U_sol_tr , dt_mod_i = pend_trapz(q0 = q0, T = T, N = N)
    Q_tr, V_tr, U_tr = newtrap_interpolated_array(X_sol_tr[:,0], X_sol_tr[:,1], U_sol_tr,
                                                     T/N, t_array_interp, 'trapz')
    
    X_sol_rk4, U_sol_rk4 , dt_rk4_i = pend_RK4(q0 = 2, T = T, N = N)
    Q_rk4, V_rk4, U_rk4 = newtrap_interpolated_array(X_sol_rk4[:,0], X_sol_rk4[:,1], U_sol_rk4,
                                                     T/N, t_array_interp, 'hermite')


    Q_exact = exact_pendulum(np.linspace(0, T, N_interp), q0)
    
    eqm_trapz_int.append(np.mean((Q_exact - Q_tr)**2))
    eqm_mod_int.append(np.mean((Q_exact - Q_mod)**2))
    eqm_rk4_int.append(np.mean((Q_exact - Q_rk4)**2))
    
    dt_trapz_int.append(dt_trapz_i)
    dt_mod_int.append(dt_mod_i)
    dt_rk4_int.append(dt_rk4_i)
    
    err_trans_trapz.append(error_transcr_medio(Q_tr, V_tr, U_tr, h))
    err_trans_mod.append(error_transcr_medio(Q_mod, V_mod, U_mod, h))
    err_trans_rk4.append(error_transcr_medio(Q_rk4, V_rk4, U_rk4, h))
    
    print(f' T = {T}, N = {N}, err = {error_transcr_medio(Q_mod, V_mod, U_mod, h)}')
    if plots:
        plot_comparison(np.linspace(0,T,N_interp),
                   Q_exact,
                   Q_tr,
                   Q_mod,
                   Q_rk4)
        
        plt.plot(np.linspace(0, T, N+1), X_sol_tr[:,0], 'k.')
        plt.plot(np.linspace(0, T, N+1), X_sol_mod[:,0], 'k.')
        plt.plot(np.linspace(0, T, N+1), X_sol_rk4[:,0], 'k.')


In [ ]:
plt.figure(figsize=[15,10])
h_scales = np.array(timescales_int)/N
plt.plot(h_scales, eqm_trapz_int, label = 'normal trapezoid')
plt.plot(h_scales, eqm_mod_int, label = 'modified trapezoid')
plt.plot(h_scales, eqm_rk4_int, label = 'RK4')
plt.vlines(h_scales, np.min(eqm_rk4_int)*0.9, np.max(eqm_trapz_int)*1.1, 'k', 'dotted')
plt.hlines(4, min(h_scales), max(h_scales), 'k', 'dotted')
plt.legend()
plt.grid()
plt.yscale('log')
plt.xscale('log')
plt.xlabel('h')
plt.ylabel('Error cuadrático medio')

In [ ]:
plt.figure(figsize=[15,10])
h_scales = np.array(timescales_int)/N
plt.plot(h_scales, err_trans_trapz, label = 'normal trapezoid')
plt.plot(h_scales, err_trans_mod, label = 'modified trapezoid')
plt.plot(h_scales, err_trans_rk4, label = 'RK4')
plt.vlines(h_scales, np.min(err_trans_rk4)*0.9, np.max(err_trans_trapz)*1.1, 'k', 'dotted')
#plt.hlines(4, min(h_scales), max(h_scales), 'k', 'dotted')
plt.legend()
plt.grid()
plt.yscale('log')
plt.xscale('log')
plt.xlabel('h')
plt.ylabel('Error de transcripción medio')

In [ ]:
plt.figure(figsize=[15,10])
h_scales = np.array(timescales_int)/N
plt.plot(h_scales, dt_trapz_int, label = 'normal trapezoid')
plt.plot(h_scales, dt_mod_int, label = 'modified trapezoid')
plt.plot(h_scales, dt_rk4_int, label = 'RK4')
plt.vlines(h_scales, np.min(dt_rk4_int)*0.9, np.max(dt_mod_int)*1.1, 'k', 'dotted')
plt.legend()
plt.grid()
plt.yscale('log')
plt.xscale('log')
plt.xlabel('h')
plt.ylabel('Tiempo de proceso (s)')

In [ ]:
T_test = 0.1
N_test = 15
X_sol_mod, U_sol_mod, _ = pend_trapz_mod(q0 = 2, T = T_test, N = N_test)
X_sol_tr, U_sol_tr, _ = pend_trapz(q0 = 2, T = T_test, N = N_test)

In [ ]:
N_interp = 1000
t_array_interp = np.linspace(0, T_test, N_interp)
h = t_array_interp[1]-t_array_interp[0]

Q_mod, V_mod, U_mod = newtrap_interpolated_array(X_sol_mod[:,0], X_sol_mod[:,1], U_sol_mod,
                                                     T_test/N_test, t_array_interp, 'mod')
Q_tr, V_tr, U_tr = newtrap_interpolated_array(X_sol_tr[:,0], X_sol_tr[:,1], U_sol_tr,
                                                     T_test/N_test, t_array_interp, 'trapz')

In [ ]:
eps_Q_mod, eps_V_mod = error_transcr(Q_mod, V_mod, np.zeros_like(Q_mod), h)
eps_Q_tr, eps_V_tr = error_transcr(Q_tr, V_tr, np.zeros_like(Q_tr), h)
plt.figure(figsize=[12,8])
plt.plot(t_array_interp[1:-1], eps_Q_mod)
plt.plot(t_array_interp[1:-1], eps_Q_tr, '--')
plt.plot(t_array_interp[1:-1], eps_V_mod)

### Comparando matrices

In [ ]:
N=15

In [ ]:
opti = Opti()
opti.solver('ipopt')

Q = opti.variable(N+1)
V = opti.variable(N+1)
U = opti.variable(N+1)
T = opti.parameter()
u_m = opti.parameter()
#t_m = opti.parameter()

cost = sum1(cos(Q)) #**2
#cost = T
#cost = -sum1(X[:,0])
opti.minimize(cost)

opti.subject_to(Q[0] == 0)
opti.subject_to(V[0] == 0)
opti.subject_to(V[-1] == 0)


for ii in range(N):
    opti.subject_to(Q[ii+1] == new_q(Q[ii], Q[ii+1], V[ii], U[ii], U[ii+1], T/N))
    opti.subject_to(V[ii+1] == new_v(Q[ii], Q[ii+1], V[ii], U[ii], U[ii+1], T/N))
    opti.subject_to(opti.bounded(-u_m,U[ii],u_m))
    
opti.set_initial(Q, np.linspace(0, pi, N+1))
opti.set_initial(V, pi/N)
#opti.set_initial(T, 50)
opti.set_value(T, 25)
max_par = 0.5
opti.set_value(u_m, max_par)

sol = opti.solve()

spar_mod = sol.value(jacobian(opti.g,opti.x)).toarray()

In [ ]:
opti = Opti()
opti.solver('ipopt')

Q = opti.variable(N+1)
V = opti.variable(N+1)
U = opti.variable(N+1)
T = opti.parameter()
u_m = opti.parameter()
#t_m = opti.parameter()

cost = sum1(cos(Q)) #**2
#cost = T
#cost = -sum1(X[:,0])
opti.minimize(cost)

opti.subject_to(Q[0] == 0)
opti.subject_to(V[0] == 0)
opti.subject_to(V[-1] == 0)


for ii in range(N):
    opti.subject_to(Q[ii+1] == new_q_tr(Q[ii], V[ii], V[ii+1], U[ii], U[ii+1], T/N))
    opti.subject_to(V[ii+1] == new_v_tr(Q[ii], Q[ii+1], V[ii], U[ii], U[ii+1], T/N))
    opti.subject_to(opti.bounded(-u_m,U[ii],u_m))
    
opti.set_initial(Q, np.linspace(0, pi, N+1))
opti.set_initial(V, pi/N)
#opti.set_initial(T, 50)
opti.set_value(T, 25)
max_par = 0.5
opti.set_value(u_m, max_par)

sol = opti.solve()
spar_tr = sol.value(jacobian(opti.g,opti.x)).toarray()

In [ ]:
opti = Opti()
opti.solver('ipopt')

Q = opti.variable(N+1)
V = opti.variable(N+1)
U = opti.variable(N+1)
T = opti.parameter()
u_m = opti.parameter()
#t_m = opti.parameter()

cost = sum1(cos(Q)) #**2
#cost = T
#cost = -sum1(X[:,0])
opti.minimize(cost)

opti.subject_to(Q[0] == 0)
opti.subject_to(V[0] == 0)
opti.subject_to(V[-1] == 0)


for ii in range(N):
    opti.subject_to(Q[ii+1] == new_q_tr_2(Q[ii], Q[ii+1], V[ii], U[ii], U[ii+1], T/N))
    opti.subject_to(V[ii+1] == new_v_tr_2(Q[ii], Q[ii+1], V[ii], U[ii], U[ii+1], T/N))
    opti.subject_to(opti.bounded(-u_m,U[ii],u_m))
    
opti.set_initial(Q, np.linspace(0, pi, N+1))
opti.set_initial(V, pi/N)
#opti.set_initial(T, 50)
opti.set_value(T, 25)
max_par = 0.5
opti.set_value(u_m, max_par)

sol = opti.solve()
spar_tr_2 = sol.value(jacobian(opti.g,opti.x)).toarray()

In [ ]:
plt.spy(spar_mod)
plt.title('Modified trapezoidal\n')

In [ ]:
plt.spy(spar_tr)

In [ ]:
plt.spy(spar_tr_2)

In [ ]:
np.count_nonzero(spar_mod), np.count_nonzero(spar_tr)

In [ ]:
spar_mod.size, spar_tr.size

## Hermite Simpson

In [ ]:
def hs_opti_step(x_n, x, u, u_n, dt):
    f = u-np.sin(x[0])
    f_n = u_n-np.sin(x_n[0])
    u_c = (u + u_n)/2
    q_c = (x[0]+x_n[0])/2 + dt/8*(x[1]-x_n[1])
    v_c = (x[1]+x_n[1])/2 + dt/8*(f-f_n)
    f_c = u_c-np.sin(q_c)
    res = np.zeros_like(x_n)
    res[1] = x[1] + dt/6*(f + 4*f_c + f_n) - x_n[1]
    res[0] = x[0] + dt/6*(x[1] + 4*v_c + x_n[1]) - x_n[0]
    return res

def hs_step(x, u, u_n, dt):
    x_n = root(hs_opti_step, x, (x, u, u_n, dt))
    return x_n.x

def integrate_hs(x_0, u, dt):
    x = [x_0,]
    u = np.append(u, u[-1])
    for ii in range(len(u)-1):
        x_i = hs_step(x[-1], u[ii], u[ii+1], dt)
        x.append(x_i)
    return horzcat(*x).T

def pend_HS(q0 = 1, T = 20, N = 50):
    t0 = time.time()
    xx = integrate_hs(np.array([q0,0.]), np.zeros(N), T/N)
    t1 = time.time()
    return np.array(xx), np.zeros(N), t1-t0

In [ ]:
q = MX.sym('q')
v = MX.sym('v')
t = MX.sym('t')
dt = MX.sym('dt')
u = MX.sym('u')

rhs = u-sin(q)
F1d = Function('F', [q, u], [rhs])
f = F1d(q,u)


q_n = MX.sym('q_n')
u_n = MX.sym('u_n')
v_n = MX.sym('v_n')
f_n = F1d(q_n,u_n)

q_c = (q + q_n)/2 + dt/8 * (v - v_n)
v_c = (v + v_n)/2 + dt/8 * (f - f_n)
u_c = (u + u_n)/2
f_c = F1d(q_c,u_c)

v_n_expr = v + (1/6) * dt * (f + 4*f_c + f_n)
q_n_expr = q + (1/6) * dt * (v + 4*v_c + v_n)

new_q_HS = Function('New_q', [q, q_n, v, v_n, u, u_n, dt], [q_n_expr])
new_v_HS = Function('New_v', [q, q_n, v, v_n, u, u_n, dt], [v_n_expr])

In [ ]:
opti = Opti()
opti.solver('ipopt')
N = 100
Q = opti.variable(N+1)
V = opti.variable(N+1)
U = opti.variable(N+1)
T = opti.parameter()
u_m = opti.parameter()
#t_m = opti.parameter()

cost = sum1(cos(Q)) #**2
#cost = T
#cost = -sum1(X[:,0])
opti.minimize(cost)

opti.subject_to(Q[0] == 0)
opti.subject_to(V[0] == 0)
opti.subject_to(V[-1] == 0)


for ii in range(N):
    opti.subject_to(Q[ii+1] == new_q_HS(Q[ii], Q[ii+1], V[ii], V[ii+1], U[ii], U[ii+1], T/N))
    opti.subject_to(V[ii+1] == new_v_HS(Q[ii], Q[ii+1], V[ii], V[ii+1], U[ii], U[ii+1], T/N))
    opti.subject_to(opti.bounded(-u_m,U[ii],u_m))


In [ ]:
opti.set_initial(Q, np.linspace(0, pi, N+1))
opti.set_initial(V, pi/N)
#opti.set_initial(T, 50)
opti.set_value(T, 30)
max_par = 0.2
opti.set_value(u_m, max_par)

In [ ]:
sol = opti.solve()

In [ ]:
X_sol = np.zeros([N+1,2])
X_sol[:,0] = sol.value(Q)
X_sol[:,1] = sol.value(V)
plot_results(X_sol, sol.value(U)[:-1], sol.value(T), max_par, N)

In [ ]:
def pend_HS_casadi(q0 = 1, T = 20, N = 50):
    opti = Opti()
    opti.solver('ipopt')

    Q = opti.variable(N+1)
    V = opti.variable(N+1)
    U = opti.variable(N+1)
    u_m = opti.parameter()

    cost = sum1(U**2) 
    opti.minimize(cost)

    opti.subject_to(Q[0] == q0)
    opti.subject_to(V[0] == 0)


    for ii in range(N):
        opti.subject_to(Q[ii+1] == new_q_HS(Q[ii], Q[ii+1], V[ii], V[ii+1], U[ii], U[ii+1], T/N))
        opti.subject_to(V[ii+1] == new_v_HS(Q[ii], Q[ii+1], V[ii], V[ii+1], U[ii], U[ii+1], T/N))
        opti.subject_to(opti.bounded(-u_m,U[ii],u_m))
        
    xx = integrate_rk4(DM([q0,0]), np.zeros(N), T/N)
    opti.set_initial(Q, xx[:,0])
    opti.set_initial(V, xx[:,1])
    opti.set_initial(U, 0)
    max_par = 0.0
    opti.set_value(u_m, max_par)
    
    t0 = time.time()
    sol_tr = opti.solve()
    t1 = time.time()
    
    X_sol_tr = np.zeros([N+1,2])
    X_sol_tr[:,0] = sol_tr.value(Q)
    X_sol_tr[:,1] = sol_tr.value(V)
    return X_sol_tr, sol_tr.value(U), t1-t0

In [ ]:
T_test = 20
N_test = 30
X_sol_HS, U_sol_HS, _ = pend_HS(q0 = 2, T = T_test, N = N_test)

In [ ]:
plot_results(X_sol_HS, U_sol_HS, T_test, 0, N_test)
plt.title('Hermite-Simpson')

## Hermite Simpson Modificado

In [ ]:
def hs_mod_opti_step(x_n, x, u, u_n, dt):
    f = u-np.sin(x[0])
    f_n = u_n-np.sin(x_n[0])
    u_c = (u + u_n)/2
    q_c = (13*x[0] + 3*x_n[0])/16 + 5*dt/16*x[1] + dt**2/96 * (4*f - f_n)
    v_c = (x[1]+x_n[1])/2 + dt/8*(f-f_n)
    f_c = u_c-np.sin(q_c)
    res = np.zeros_like(x_n)
    res[1] = x[1] + dt/6*(f + 4*f_c + f_n) - x_n[1]
    res[0] = x[0] + dt*x[1] + dt**2/6*(f + 2*f_c) - x_n[0]
    return res

def hs_mod_step(x, u, u_n, dt):
    x_n = root(hs_mod_opti_step, x, (x, u, u_n, dt))
    return x_n.x

def integrate_hs_mod(x_0, u, dt):
    x = [x_0,]
    u = np.append(u, u[-1])
    for ii in range(len(u)-1):
        x_i = hs_mod_step(x[-1], u[ii], u[ii+1], dt)
        x.append(x_i)
    return horzcat(*x).T

def pend_HS_mod(q0 = 1, T = 20, N = 50):
    t0 = time.time()
    xx = integrate_hs_mod(np.array([q0,0.]), np.zeros(N), T/N)
    t1 = time.time()
    return np.array(xx), np.zeros(N), t1-t0

In [ ]:
q = MX.sym('q')
v = MX.sym('v')
t = MX.sym('t')
dt = MX.sym('dt')
u = MX.sym('u')

rhs = u-sin(q)
F1d = Function('F', [q, u], [rhs])
f = F1d(q,u)


q_n = MX.sym('q_n')
u_n = MX.sym('u_n')
v_n = MX.sym('v_n')
f_n = F1d(q_n,u_n)

q_c = (13*q + 3*q_n + 5*v*dt)/16 + dt**2/96 * (4*f - f_n)
v_c = (v + v_n)/2 + dt/8 * (f - f_n)
u_c = (u + u_n)/2
f_c = F1d(q_c,u_c)

v_n_expr = v + (1/6) * dt * (f + 4*f_c + f_n)
q_n_expr = q + v * dt + (1/6) * dt**2 * (f + 2*f_c)

new_q_HS_mod = Function('New_q', [q, q_n, v, v_n, u, u_n, dt], [q_n_expr])
new_v_HS_mod = Function('New_v', [q, q_n, v, v_n, u, u_n, dt], [v_n_expr])

In [ ]:
opti = Opti()
opti.solver('ipopt')
N = 100
Q = opti.variable(N+1)
V = opti.variable(N+1)
U = opti.variable(N+1)
T = opti.parameter()
u_m = opti.parameter()
#t_m = opti.parameter()

cost = sum1(cos(Q)) #**2
#cost = T
#cost = -sum1(X[:,0])
opti.minimize(cost)

opti.subject_to(Q[0] == 0)
opti.subject_to(V[0] == 0)
opti.subject_to(V[-1] == 0)


for ii in range(N):
    opti.subject_to(Q[ii+1] == new_q_HS_mod(Q[ii], Q[ii+1], V[ii], V[ii+1], U[ii], U[ii+1], T/N))
    opti.subject_to(V[ii+1] == new_v_HS_mod(Q[ii], Q[ii+1], V[ii], V[ii+1], U[ii], U[ii+1], T/N))
    opti.subject_to(opti.bounded(-u_m,U[ii],u_m))


In [ ]:
opti.set_initial(Q, np.linspace(0, pi, N+1))
opti.set_initial(V, pi/N)
#opti.set_initial(T, 50)
opti.set_value(T, 30)
max_par = 0.2
opti.set_value(u_m, max_par)

In [ ]:
sol = opti.solve()

In [ ]:
X_sol = np.zeros([N+1,2])
X_sol[:,0] = sol.value(Q)
X_sol[:,1] = sol.value(V)
plot_results(X_sol, sol.value(U)[:-1], sol.value(T), max_par, N)

In [ ]:
def pend_HS_mod_casadi(q0 = 1, T = 20, N = 50):
    opti = Opti()
    opti.solver('ipopt')

    Q = opti.variable(N+1)
    V = opti.variable(N+1)
    U = opti.variable(N+1)
    u_m = opti.parameter()

    cost = sum1(U**2) 
    opti.minimize(cost)

    opti.subject_to(Q[0] == q0)
    opti.subject_to(V[0] == 0)


    for ii in range(N):
        opti.subject_to(Q[ii+1] == new_q_HS_mod(Q[ii], Q[ii+1], V[ii], V[ii+1], U[ii], U[ii+1], T/N))
        opti.subject_to(V[ii+1] == new_v_HS_mod(Q[ii], Q[ii+1], V[ii], V[ii+1], U[ii], U[ii+1], T/N))
        opti.subject_to(opti.bounded(-u_m,U[ii],u_m))
        
    xx = integrate_rk4(DM([q0,0]), np.zeros(N), T/N)
    opti.set_initial(Q, xx[:,0])
    opti.set_initial(V, xx[:,1])
    opti.set_initial(U, 0)
    max_par = 0.0
    opti.set_value(u_m, max_par)
    
    t0 = time.time()
    sol_tr = opti.solve()
    t1 = time.time()
    
    X_sol_tr = np.zeros([N+1,2])
    X_sol_tr[:,0] = sol_tr.value(Q)
    X_sol_tr[:,1] = sol_tr.value(V)
    return X_sol_tr, sol_tr.value(U), t1-t0

In [ ]:
T_test = 20
N_test = 30
X_sol_HS_mod, U_sol_HS_mod, _ = pend_HS_mod(q0 = 2, T = T_test, N = N_test)

In [ ]:
plot_results(X_sol_HS_mod, U_sol_HS_mod, T_test, 0, N_test)
plt.title('Hermite-Simpson modificado')

In [ ]:
def new_trap_interp_q(q, q_n, v, u, u_n, tau, h):
    res = q + v * tau + 1/2 * F1d(q,u) * tau**2 + 1/(6*h) * tau**3 * (F1d(q_n,u_n) - F1d(q,u))
    return res
def trap_interp_q(q, v, v_n, tau, h):
    res = q + v * tau + 1/(2*h) * tau**2 * (v_n - v)
    return res
def new_trap_interp_v(q, q_n, v, u, u_n, tau, h):
    res = v + tau * F1d(q,u) + tau**2 / (2*h) * (F1d(q_n,u_n) - F1d(q,u))
    return res
def hs_interp_v(q, q_n, q_c, v, v_n, v_c, u, u_n, u_c, tau, h):
    f = F1d(q,u)
    f_c = F1d(q_c,u_c)
    f_n = F1d(q_n,u_n)
    res = v + f*tau + 1/2 * (-3*f + 4*f_c - f_n)*tau**2/h + 1/3 *(2*f - 4*f_c + 2*f_n)*tau**3/(h**2)
    return res
def hs_interp_q(q, q_n, q_c, v, v_n, v_c, u, u_n, u_c, tau, h):
    res = q + v*tau + 1/2 * (-3*v + 4*v_c - v_n)*tau**2/h + 1/3 *(2*v - 4*v_c + 2*v_n)*tau**3/(h**2)
    return res
def hs_midpoint(q, q_n, v, v_n, u, u_n, tau, h):
    f = F1d(q,u)
    f_n = F1d(q_n,u_n)
    v_c = (v + v_n)/2 + h/8*(f - f_n)
    q_c = (q + q_n)/2 + h/8*(v - v_n)
    return q_c, v_c
    return res
def hs_mod_interp_q(q, q_n, q_c, v, v_n, v_c, u, u_n, u_c, tau, h):
    f = F1d(q,u)
    f_c = F1d(q_c,u_c)
    f_n = F1d(q_n,u_n)
    res = q + v*tau + 1/2 * f * tau**2 + 1/6 * (-3*f + 4*f_c - f_n)*tau**3/h 
    res = res + 1/12 *(2*f - 4*f_c + 2*f_n)*tau**4/(h**2)
    return res
def hs_mod_midpoint(q, q_n, v, v_n, u, u_n, tau, h):
    f = F1d(q,u)
    f_n = F1d(q_n,u_n)
    v_c = (v + v_n)/2 + h/8*(f - f_n)
    q_c = (13*q + 3*q_n + 5*v*h)/16 + h**2/96 * (4*f - f_n)
    return q_c, v_c
def newpoint(Q, V, U, h, t, scheme):
    n = int(t//h)
    tau = t%h
    #print(f't = {t} , tau = {tau} , n = {n} , h = {h}')
    if abs(tau) < h*Q.size*1e-10:
        q = Q[n]
        v = V[n]
    else:
        if scheme == 'mod':
            q = new_trap_interp_q(Q[n], Q[n+1], V[n], U[n], U[n+1], tau, h)
            v = new_trap_interp_v(Q[n], Q[n+1], V[n], U[n], U[n+1], tau, h)
        elif scheme == 'trapz':
            q = trap_interp_q(Q[n], V[n], V[n+1], tau, h)
            v = new_trap_interp_v(Q[n], Q[n+1], V[n], U[n], U[n+1], tau, h)
        elif scheme == 'hs':
            q_c, v_c = hs_midpoint(Q[n], Q[n+1], V[n], V[n+1], U[n], U[n+1], tau, h)
            u_c = (U[n] + U[n+1])/2
            q = hs_interp_q(Q[n], Q[n+1], q_c, V[n], V[n+1], v_c, U[n], U[n+1], u_c, tau, h)
            v = hs_interp_v(Q[n], Q[n+1], q_c, V[n], V[n+1], v_c, U[n], U[n+1], u_c, tau, h)
        elif scheme == 'hsmod':
            q_c, v_c = hs_mod_midpoint(Q[n], Q[n+1], V[n], V[n+1], U[n], U[n+1], tau, h)
            u_c = (U[n] + U[n+1])/2
            q = hs_mod_interp_q(Q[n], Q[n+1], q_c, V[n], V[n+1], v_c, U[n], U[n+1], u_c, tau, h)
            v = hs_interp_v(Q[n], Q[n+1], q_c, V[n], V[n+1], v_c, U[n], U[n+1], u_c, tau, h)
        else:
            raise NameError(f'scheme {scheme} not recognized')
    return q,v
def newtrap_interpolated_array(Q, V, U, h, t_array, scheme = 'mod'):
    N = t_array.size
    new_Q = np.zeros(N)
    new_V = np.zeros(N)
    if Q.size != U.size:
        U = np.append(U, U[-1])
    old_t_array = np.linspace(0, (Q.size-1)*h, Q.size)
    #print(Q.size, U.size, old_t_array.size, Q, U, old_t_array)
    new_U = np.interp(t_array, old_t_array, U)
    if scheme == 'hermite':
        q_interp = hermite(old_t_array, Q, V)
        new_Q = q_interp(t_array)
        A = U-np.sin(Q)
        v_interp = hermite(old_t_array, V, A)
        new_V = v_interp(t_array)
    else:
        for ii in range(N):
            new_Q[ii], new_V[ii] = newpoint(Q, V, U, h, t_array[ii], scheme)
    return new_Q, new_V, new_U

In [ ]:
T_test = 20
N_test = 10
X_sol_HS, U_sol_HS, _ = pend_HS(q0 = 2, T = T_test, N = N_test)

In [ ]:
N_interp = 400
t_array_interp = np.linspace(0, 20, N_interp)
new_Q, new_V, new_U = newtrap_interpolated_array(X_sol_HS[:,0], X_sol_HS[:,1], U_sol_HS,
                                                 T_test/N_test, t_array_interp, 'hs')
X_interp = np.zeros([N_interp,2])
X_interp[:,0] = new_Q
X_interp[:,1] = new_V
plot_results(X_interp, new_U[:-1], t_array_interp[-1], max_par, N_interp-1, figsize = [20,15])
plt.plot(np.linspace(0, T_test, N_test+1), X_sol_HS[:,0], 'k.')
plt.plot(np.linspace(0, T_test, N_test+1), X_sol_HS[:,1], 'k.')

In [ ]:
T_test = 20
N_test = 10
X_sol_HS_mod, U_sol_HS_mod, _ = pend_HS_mod(q0 = 2, T = T_test, N = N_test)

In [ ]:
N_interp = 400
t_array_interp = np.linspace(0, 20, N_interp)
new_Q, new_V, new_U = newtrap_interpolated_array(X_sol_HS_mod[:,0], X_sol_HS_mod[:,1], U_sol_HS_mod,
                                                 T_test/N_test, t_array_interp, 'hsmod')
X_interp = np.zeros([N_interp,2])
X_interp[:,0] = new_Q
X_interp[:,1] = new_V
plot_results(X_interp, new_U[:-1], t_array_interp[-1], max_par, N_interp-1, figsize = [20,15])
plt.plot(np.linspace(0, T_test, N_test+1), X_sol_HS_mod[:,0], 'k.')
plt.plot(np.linspace(0, T_test, N_test+1), X_sol_HS_mod[:,1], 'k.')

In [ ]:
timescales = [0.2, 0.5, 1, 2, 5, 7, 10, 25, 50, 70, 80, 90, 100, 120, 150]

eqm_hs = []
eqm_hs_mod = []
eqm_rk4 = []

dt_hs = []
dt_hs_mod = []
dt_rk4 = []

N = 100
q0 = 2
plots = False
for T in timescales:
    X_sol_hs_mod, _ , dt_hs_mod_i = pend_HS_mod(q0 = q0, T = T, N = N)
    X_sol_hs, _ , dt_hs_i = pend_HS(q0 = q0, T = T, N = N)
    X_sol_rk4, _ , dt_rk4_i = pend_RK4(q0 = q0, T = T, N = N)
    Q_exact = exact_pendulum(np.linspace(0, T, N+1), q0)
    
    eqm_hs.append(np.mean((Q_exact - X_sol_hs[:,0])**2))
    eqm_hs_mod.append(np.mean((Q_exact - X_sol_hs_mod[:,0])**2))
    eqm_rk4.append(np.mean((Q_exact - X_sol_rk4[:,0])**2))
    
    dt_hs.append(dt_hs_i)
    dt_hs_mod.append(dt_hs_mod_i)
    dt_rk4.append(dt_rk4_i)
    
    if plots:
        plot_comparison(np.linspace(0,T,N+1),
                   Q_exact,
                   X_sol_hs[:,0],
                   X_sol_hs_mod[:,0],
                   X_sol_rk4[:,0])


In [ ]:
plt.figure(figsize=[15,10])
h_scales = np.array(timescales)/N
plt.plot(h_scales, eqm_hs, label = 'normal Hermite Simpson')
plt.plot(h_scales, eqm_hs_mod, label = 'modified Hermite Simpson')
plt.plot(h_scales, eqm_rk4, label = 'RK4')
plt.vlines(h_scales, np.min(eqm_rk4)*0.9, np.max(eqm_trapz)*1.1, 'k', 'dotted')
plt.hlines(4, min(h_scales), max(h_scales), 'k', 'dotted')
plt.legend()
plt.grid()
plt.yscale('log')
plt.xscale('log')
plt.xlabel('h')
plt.ylabel('Error cuadrático medio')

In [ ]:
plt.figure(figsize=[15,10])
oldtimescales = [0.2, 0.5, 0.6, 0.7, 1, 2, 5, 10, 25, 50, 70, 80, 90, 100]
oldh_scales = np.array(oldtimescales)/N
h_scales = np.array(timescales)/N
plt.plot(h_scales, dt_hs, label = 'normal Hermite Simpson')
plt.plot(h_scales, dt_hs_mod, label = 'modified Hermite Simpson')
plt.plot(oldh_scales, dt_mod[1:], label = 'modified trapezoid')
plt.plot(h_scales, dt_rk4, label = 'RK4')
plt.vlines(h_scales, np.min(dt_rk4)*0.9, np.max(dt_hs_mod)*1.1, 'k', 'dotted')
plt.legend()
plt.grid()
plt.yscale('log')
plt.xscale('log')
plt.xlabel('h')
plt.ylabel('Tiempo de proceso (s)')

### Error de interpolaciones

In [ ]:
timescales_int = [0.1, 0.2, 0.5, 1, 2, 5, 7, 10, 12, 14, 15, 17, 20, 22, 25]

eqm_hs_int = []
eqm_hs_mod_int = []
eqm_rk4_int = []

dt_hs_int = []
dt_hs_mod_int = []
dt_rk4_int = []

err_trans_hs = []
err_trans_hs_mod = []
err_trans_rk4 = []

N = 15
q0 = 2
N_interp = 1000
plots = False

for T in timescales_int:
    
    t_array_interp = np.linspace(0, T, N_interp)
    h = t_array_interp[1]-t_array_interp[0]
    
    X_sol_hs_mod, U_sol_hs_mod , dt_hs_i = pend_HS_mod(q0 = q0, T = T, N = N)
    Q_hs_mod, V_hs_mod, U_hs_mod = newtrap_interpolated_array(X_sol_hs_mod[:,0], X_sol_hs_mod[:,1], U_sol_hs_mod,
                                                     T/N, t_array_interp, 'hsmod')
    #print(U_sol_hs_mod)
    X_sol_hs, U_sol_hs, dt_hs_mod_i = pend_HS(q0 = q0, T = T, N = N)
    Q_hs, V_hs, U_hs = newtrap_interpolated_array(X_sol_hs[:,0], X_sol_hs[:,1], U_sol_hs,
                                                     T/N, t_array_interp, 'hs')
    
    X_sol_rk4, U_sol_rk4 , dt_rk4_i = pend_RK4(q0 = 2, T = T, N = N)
    Q_rk4, V_rk4, U_rk4 = newtrap_interpolated_array(X_sol_rk4[:,0], X_sol_rk4[:,1], U_sol_rk4,
                                                     T/N, t_array_interp, 'hermite')


    Q_exact = exact_pendulum(np.linspace(0, T, N_interp), q0)
    
    eqm_hs_int.append(np.mean((Q_exact - Q_hs)**2))
    eqm_hs_mod_int.append(np.mean((Q_exact - Q_hs_mod)**2))
    eqm_rk4_int.append(np.mean((Q_exact - Q_rk4)**2))
    
    dt_hs_int.append(dt_hs_i)
    dt_hs_mod_int.append(dt_hs_mod_i)
    dt_rk4_int.append(dt_rk4_i)
    
    err_trans_hs.append(error_transcr_medio(Q_hs, V_hs, U_hs, h))
    err_trans_hs_mod.append(error_transcr_medio(Q_hs_mod, V_hs_mod, U_hs_mod, h))
    err_trans_rk4.append(error_transcr_medio(Q_rk4, V_rk4, U_rk4, h))
    
    print(f' T = {T}, N = {N}, err = {error_transcr_medio(Q_hs_mod, V_hs_mod, U_hs_mod, h)}')
    if plots:
        plot_comparison(np.linspace(0,T,N_interp),
                   Q_exact,
                   Q_hs,
                   Q_hs_mod,
                   Q_rk4)
        
        plt.plot(np.linspace(0, T, N+1), X_sol_hs[:,0], 'k.')
        plt.plot(np.linspace(0, T, N+1), X_sol_hs_mod[:,0], 'k.')
        plt.plot(np.linspace(0, T, N+1), X_sol_rk4[:,0], 'k.')


In [ ]:
plt.figure(figsize=[15,10])
h_scales = np.array(timescales_int)/N
plt.plot(h_scales, eqm_hs_int, label = 'Hermite Simpson')
plt.plot(h_scales, eqm_hs_mod_int, label = 'modified Hermite Simpson')
plt.plot(h_scales, eqm_rk4_int, label = 'RK4')
plt.vlines(h_scales, np.min(eqm_hs_mod_int)*0.9, np.max(eqm_rk4_int)*1.1, 'k', 'dotted')
plt.hlines(4, min(h_scales), max(h_scales), 'k', 'dotted')
plt.legend()
plt.grid()
plt.yscale('log')
plt.xscale('log')
plt.xlabel('h')
plt.ylabel('Error cuadrático medio')

In [ ]:
plt.figure(figsize=[15,10])
h_scales = np.array(timescales_int)/N
plt.plot(h_scales, err_trans_hs, label = 'Hermite Simpson')
plt.plot(h_scales, err_trans_hs_mod, label = 'modified Hermite Simpson')
plt.plot(h_scales, err_trans_rk4, label = 'RK4')
plt.vlines(h_scales, np.min(err_trans_rk4)*0.9, np.max(err_trans_trapz)*1.1, 'k', 'dotted')
#plt.hlines(4, min(h_scales), max(h_scales), 'k', 'dotted')
plt.legend()
plt.grid()
plt.yscale('log')
plt.xscale('log')
plt.xlabel('h')
plt.ylabel('Error de transcripción medio')

In [ ]:
plt.figure(figsize=[15,10])
h_scales = np.array(timescales_int)/N
plt.plot(h_scales, dt_hs_int, label = 'normal Hermite Simpson')
plt.plot(h_scales, dt_hs_mod_int, label = 'modified Hermite Simpson')
plt.plot(h_scales, dt_mod_int, label = 'modified trapezoid')
plt.plot(h_scales, dt_rk4_int, label = 'RK4')
plt.vlines(h_scales, np.min(dt_rk4_int)*0.9, np.max(dt_mod_int)*1.1, 'k', 'dotted')
plt.legend()
plt.grid()
plt.yscale('log')
plt.xscale('log')
plt.xlabel('h')
plt.ylabel('Tiempo de proceso (s)')

In [ ]:
N=15

In [ ]:
opti = Opti()
opti.solver('ipopt')

Q = opti.variable(N+1)
V = opti.variable(N+1)
U = opti.variable(N+1)
T = opti.parameter()
u_m = opti.parameter()
#t_m = opti.parameter()

cost = sum1(cos(Q)) #**2
#cost = T
#cost = -sum1(X[:,0])
opti.minimize(cost)

opti.subject_to(Q[0] == 0)
opti.subject_to(V[0] == 0)
opti.subject_to(V[-1] == 0)


for ii in range(N):
    opti.subject_to(Q[ii+1] == new_q_HS(Q[ii], Q[ii+1], V[ii], V[ii+1], U[ii], U[ii+1], T/N))
    opti.subject_to(V[ii+1] == new_v_HS(Q[ii], Q[ii+1], V[ii], V[ii+1], U[ii], U[ii+1], T/N))
    opti.subject_to(opti.bounded(-u_m,U[ii],u_m))
    
opti.set_initial(Q, np.linspace(0, pi, N+1))
opti.set_initial(V, pi/N)
#opti.set_initial(T, 50)
opti.set_value(T, 25)
max_par = 0.5
opti.set_value(u_m, max_par)

sol = opti.solve()

spar_hs = sol.value(jacobian(opti.g,opti.x)).toarray()

In [ ]:
opti = Opti()
opti.solver('ipopt')

Q = opti.variable(N+1)
V = opti.variable(N+1)
U = opti.variable(N+1)
T = opti.parameter()
u_m = opti.parameter()
#t_m = opti.parameter()

cost = sum1(cos(Q)) #**2
#cost = T
#cost = -sum1(X[:,0])
opti.minimize(cost)

opti.subject_to(Q[0] == 0)
opti.subject_to(V[0] == 0)
opti.subject_to(V[-1] == 0)


for ii in range(N):
    opti.subject_to(Q[ii+1] == new_q_HS_mod(Q[ii], Q[ii+1], V[ii], V[ii+1], U[ii], U[ii+1], T/N))
    opti.subject_to(V[ii+1] == new_v_HS_mod(Q[ii], Q[ii+1], V[ii], V[ii+1], U[ii], U[ii+1], T/N))
    opti.subject_to(opti.bounded(-u_m,U[ii],u_m))
    
opti.set_initial(Q, np.linspace(0, pi, N+1))
opti.set_initial(V, pi/N)
#opti.set_initial(T, 50)
opti.set_value(T, 25)
max_par = 0.5
opti.set_value(u_m, max_par)

sol = opti.solve()

spar_hs_mod = sol.value(jacobian(opti.g,opti.x)).toarray()

In [ ]:
plt.spy(spar_hs_mod)
plt.title('modified Hermite Simpson\n')

In [ ]:
plt.spy(spar_hs)
plt.title('Normal Hermite Simpson\n')

### Comparación

In [ ]:
T_test = 20
N_test = 12
X_sol_HS_mod, U_sol_HS_mod, _ = pend_HS_mod(q0 = 2, T = T_test, N = N_test)
X_sol_HS, U_sol_HS, _ = pend_HS(q0 = 2, T = T_test, N = N_test)
X_sol_rk4, U_sol_rk4, _ = pend_RK4(q0 = 2, T = T_test, N = N_test)

In [ ]:
N_interp = 500
t_array_interp = np.linspace(0, T_test, N_interp)

new_Q, new_V, new_U = newtrap_interpolated_array(X_sol_HS_mod[:,0], X_sol_HS_mod[:,1], U_sol_HS_mod,
                                                 T_test/N_test, t_array_interp, 'hsmod')
X_interp_HS_mod = np.zeros([N_interp,2])
X_interp_HS_mod[:,0] = new_Q
X_interp_HS_mod[:,1] = new_V

new_Q, new_V, new_U = newtrap_interpolated_array(X_sol_HS[:,0], X_sol_HS[:,1], U_sol_HS,
                                                 T_test/N_test, t_array_interp, 'hs')
X_interp_HS = np.zeros([N_interp,2])
X_interp_HS[:,0] = new_Q
X_interp_HS[:,1] = new_V

new_Q, new_V, new_U = newtrap_interpolated_array(X_sol_rk4[:,0], X_sol_rk4[:,1], U_sol_rk4,
                                                 T_test/N_test, t_array_interp, 'hermite')
X_interp_rk4 = np.zeros([N_interp,2])
X_interp_rk4[:,0] = new_Q
X_interp_rk4[:,1] = new_V

In [ ]:
plt.figure(figsize=[15,10])
plt.plot(t_array_interp, exact_pendulum(t_array_interp, 2), 'r', label = 'exact')
plt.plot(t_array_interp, X_interp_HS_mod[:,0], label = 'modified HS')
plt.plot(t_array_interp, X_interp_HS[:,0], label = 'normal HS')
plt.plot(t_array_interp, X_interp_rk4[:,0],  label = 'RK4')
plt.plot(np.linspace(0, T_test, N_test+1), X_sol_HS[:,0], 'k.')
plt.plot(np.linspace(0, T_test, N_test+1), X_sol_HS_mod[:,0], 'k.')
plt.plot(np.linspace(0, T_test, N_test+1), exact_pendulum(np.linspace(0, T_test, N_test+1), 2), 'k.')
plt.plot(np.linspace(0, T_test, N_test+1), X_sol_rk4[:,0], 'k.')
plt.legend()
plt.grid()